In [21]:
import pandas as pd
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from sharadar.pipeline.factors import (
    MarketCap,
    EV,
    Fundamentals,
    Sector
)
from sharadar.pipeline.engine import load_sharadar_bundle, symbols, make_pipeline_engine
from zipline.pipeline.filters import StaticAssets
import time
import datetime
from sharadar.pipeline.universes import TRADABLE_STOCKS_US, NamedUniverse, TradableStocksUS

In [27]:
engine = make_pipeline_engine()

pipe_date = pd.to_datetime('2020-07-01', utc=True)

pipe = Pipeline(columns={
    'mkt_cap': MarketCap(),
    'ev': EV(),
    'netmargin': Fundamentals(field='netmargin'),
    'revenue': Fundamentals(field='revenue'),
    'evebit': Fundamentals(field='evebit'),
    'sector': Sector()
},
#screen = StaticAssets(symbols(['IBM', 'F', 'AAPL']))
screen = NamedUniverse(TRADABLE_STOCKS_US)
#screen = TradableStocksUS()
)

df = engine.run_pipeline(pipe, pipe_date)

[2020-07-03 21:53:13.739733] INFO: sharadar_db_bundle: Compute pipeline values in chunks of 120 days.
[2020-07-03 21:53:13.740576] INFO: sharadar_db_bundle: Compute values for pipeline from 2020-07-01 to 2020-07-01.
[2020-07-03 21:53:13.746409] INFO: sharadar_db_bundle: Computing term 1 of 13 [AssetExists()]
[2020-07-03 21:53:13.747258] INFO: sharadar_db_bundle: Term already in workspace: no computation needed
[2020-07-03 21:53:13.747984] INFO: sharadar_db_bundle: Computing term 2 of 13 [Fundamentals{'field': 'sharesbas_arq'}]
[2020-07-03 21:53:13.751792] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.003778
[2020-07-03 21:53:13.752980] INFO: sharadar_db_bundle: Computing term 3 of 13 [Fundamentals{'field': 'sharefactor_arq'}]
[2020-07-03 21:53:13.756230] INFO: sharadar_db_bundle: Elapsed time: 0:00:00.003250
[2020-07-03 21:53:13.757140] INFO: sharadar_db_bundle: Computing term 4 of 13 [Fundamentals{'field': 'revenue'}]
[2020-07-03 21:53:13.759375] INFO: sharadar_db_bundle: Elapsed ti

In [30]:
df.head()

ev  evebit  \
2020-07-01 00:00:00+00:00 Equity(109811 [MSGE])  1.060622e+09     NaN   
                          Equity(110003 [ARNC])  1.844107e+09     4.0   
                          Equity(110006 [CARR])  3.043305e+10     NaN   
                          Equity(110110 [RVMD])  1.690874e+09   -25.0   
                          Equity(110121 [SDGR])  5.665788e+09   -77.0   

                                                      mkt_cap  netmargin  \
2020-07-01 00:00:00+00:00 Equity(109811 [MSGE])  1.801448e+09     -0.643   
                          Equity(110003 [ARNC])  1.519107e+09      0.037   
                          Equity(110006 [CARR])  1.924605e+10      0.025   
                          Equity(110110 [RVMD])  1.863069e+09     -1.883   
                          Equity(110121 [SDGR])  5.796053e+09     -0.528   

                                                      revenue  \
2020-07-01 00:00:00+00:00 Equity(109811 [MSGE])  1.998610e+08   
                          Equity(110003 [ARNC])  1.611000e+09   
                          Equity(110006 [CARR])  3.888000e+09   
                          Equity(110110 [RVMD])  1.154600e+07   
                          Equity(110121 [SDGR])  2.617400e+07   

                                                            sector  
2020-07-01 00:00:00+00:00 Equity(109811 [MSGE])  Consumer Cyclical  
                          Equity(110003 [ARNC])        Industrials  
                          Equity(110006 [CARR])    Basic Materials  
                          Equity(110110 [RVMD])         Healthcare  
                          Equity(110121 [SDGR])         Technology

In [31]:
df_group_by_sector = df.groupby('sector')
sector_size = df_group_by_sector.size()
sector_size

sector
Basic Materials           110
Communication Services     20
Consumer Cyclical         244
Consumer Defensive        101
Energy                    101
Financial Services          0
Healthcare                302
Industrials               285
NA                          0
Real Estate                 0
Technology                360
Utilities                  62
dtype: int64

In [32]:
# There are Sectors with size less than 30?
len(sector_size[sector_size < 30]) > 0

True

In [35]:
sector_rank = df_group_by_sector.rank(ascending=False, pct=True, numeric_only=True)
sector_rank.sort_values(by='netmargin', ascending=False).head()

ev    evebit   mkt_cap  \
2020-07-01 00:00:00+00:00 Equity(197000 [OGE])   0.580645  0.983871  0.532258   
                          Equity(124124 [ASH])   0.345455  0.981651  0.336364   
                          Equity(124103 [CARS])  0.893443  0.837500  0.987705   
                          Equity(198326 [NWL])   0.336634  0.885000  0.366337   
                          Equity(194516 [CXO])   0.257426  0.648515  0.237624   

                                                 netmargin   revenue  
2020-07-01 00:00:00+00:00 Equity(197000 [OGE])         1.0  0.661290  
                          Equity(124124 [ASH])         1.0  0.518182  
                          Equity(124103 [CARS])        1.0  0.905738  
                          Equity(198326 [NWL])         1.0  0.376238  
                          Equity(194516 [CXO])         1.0  0.445545

In [41]:
score = sector_rank.mean(axis=1)
score.head()

2020-07-01 00:00:00+00:00  Equity(109811 [MSGE])    0.830943
                           Equity(110003 [ARNC])    0.601677
                           Equity(110006 [CARR])    0.212500
                           Equity(110110 [RVMD])    0.712816
                           Equity(110121 [SDGR])    0.736530
dtype: float64